Week 3 Assignment/Part 3

Explore and cluster the neighborhoods in Toronto
 

In [160]:
# !pip3 install folium
# !pip3 install sklearn
import numpy as np
import pandas as pd
import folium
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.preprocessing import StandardScaler

Load data from file.

In [2]:
neigh_df = pd.read_pickle('3.2.pkl')
neigh_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Visualize the whole Toronto neighborhood.

In [21]:
latitude = 43.7032
longitude = -79.3832
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7032, -79.3832.


In [200]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(neigh_df['Latitude'], neigh_df['Longitude'], neigh_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Define FourSquare Credentials.

In [23]:
CLIENT_ID = 'LVQ521WZ0ZVT1LNDBDI4FEC4TH5GTPFUJ3IQT21HTNJN4QUO'
CLIENT_SECRET = 'PUZJSH44LO0PW3S5BX4ISNSN1GBPBX0T55NBO4ELRQFTRAUA'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LVQ521WZ0ZVT1LNDBDI4FEC4TH5GTPFUJ3IQT21HTNJN4QUO
CLIENT_SECRET:PUZJSH44LO0PW3S5BX4ISNSN1GBPBX0T55NBO4ELRQFTRAUA


Explore the Toronto neighborhood.

Create a method to get all the venues around a specific neighborhood. Note that I did not implement this method.  
Credit to lab tutorials made by IBM.

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Apply the method above to the whole dataframe to collect all venues in the neighborhood.

In [ ]:
toronto_venues = getNearbyVenues(names=neigh_df['Neighborhood'],
                                 latitudes=neigh_df['Latitude'],
                                 longitudes=neigh_df['Longitude'])

Check the collected dataset.

In [119]:
print("Our venues dataset includes", toronto_venues.shape[0], 'venues, each has', toronto_venues.shape[1], 'attributes')
print('There are', len(toronto_venues['Venue Category'].unique()), 'categories in total')
toronto_venues.head()

Our venues dataset includes 2250 venues, each has 7 attributes
There are 277 categories in total


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Construct a feature vector. 

In [130]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(0, 'NeighborhoodName', toronto_venues['Neighborhood'])

Now, we need to group by neighborhood to compute the histogram of category for each area. I would like to use sum() instead of average(). Data standardization will come later before inputting to K-Mean.

In [143]:
toronto_grouped = toronto_onehot.groupby('NeighborhoodName').sum().reset_index()
toronto_grouped.head()

,NeighborhoodName,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

Now, I will apply it my data to get the top 10 venue categories for each neighborhood

In [173]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['NeighborhoodName']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Bar,Cosmetics Shop,Gym,Restaurant
1,Agincourt,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Coffee Shop,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Coffee Shop,Pharmacy,Fast Food Restaurant,Pizza Place,Beer Store,Sandwich Place,Doner Restaurant,Dim Sum Restaurant
4,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Skating Rink,Coffee Shop,Dance Studio,Pharmacy,Bank,Pub,Sandwich Place


Clustering using K-Mean.

In [194]:
kclusters = 3
X = toronto_grouped.drop('NeighborhoodName', 1).astype(float)
X = StandardScaler().fit_transform(X)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)
yhat = kmeans.labels_

Merging clustering result with other attributes.

In [195]:
toronto_df = pd.merge(neigh_df, neighborhoods_venues_sorted, on='Neighborhood', how='right')
toronto_df.insert(5, 'Cluster Labels', yhat)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Moving Target,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Medical Center,Rental Car Location,Electronics Store,Pizza Place,Breakfast Spot,Mexican Restaurant,Doner Restaurant,Diner,Discount Store,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Athletics & Sports,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Donut Shop,Discount Store,Dog Run


Visualizing.

In [196]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood'], toronto_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**The map might help us more understand the clusters. One can see that there is one featured red circle right at the center, around them are some purple, and the majority are green, covering all over the map.**

Cluster 0.

In [197]:
toronto_df[toronto_df['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0,Café,Bookstore,Japanese Restaurant,Restaurant,Bar,Bakery,Coffee Shop,Chinese Restaurant,Jazz Club,Sushi Restaurant


**Clearly, this area is perfectly designed for a univeristy, they have cafe, bookstore, bakery, jazz club, bar, and so on.**

Cluster 1.

In [198]:
toronto_df[toronto_df['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,1,Motel,American Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
19,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1,Cafeteria,Martial Arts Dojo,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
20,M2N,North York,Willowdale South,43.770120,-79.408493,1,Ramen Restaurant,Restaurant,Sandwich Place,Café,Pizza Place,Electronics Store,Pet Store,Indonesian Restaurant,Steakhouse,Hotel
22,M2R,North York,Willowdale West,43.782736,-79.442259,1,Grocery Store,Pharmacy,Coffee Shop,Pizza Place,Butcher,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
27,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,1,Bar,Falafel Restaurant,Coffee Shop,Massage Studio,Eastern European Restaurant,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
32,M4A,North York,Victoria Village,43.725882,-79.315572,1,Hockey Arena,Intersection,Coffee Shop,Portuguese Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
42,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Swim School,Bus Line,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Donut Shop
43,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Grocery Store,Hotel,Restaurant,Sandwich Place,Burger Joint,Breakfast Spot,Food & Drink Shop,Asian Restaurant,Park,General Entertainment
49,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,Restaurant,Coffee Shop,Pizza Place,Flower Shop,Bakery,Indian Restaurant,Italian Restaurant,Pub,Café,Beer Store


**These areas seem to be "the most livable" areas in Toronto, they have various utalities around, e.g., park, yoga, dinner, dimsum, etc.**

Cluster 2.

In [199]:
toronto_df[toronto_df['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Moving Target,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Medical Center,Rental Car Location,Electronics Store,Pizza Place,Breakfast Spot,Mexican Restaurant,Doner Restaurant,Diner,Discount Store,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Athletics & Sports,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Donut Shop,Discount Store,Dog Run
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Convenience Store,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,2,Discount Store,Department Store,Coffee Shop,Bus Station,Train Station,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Bakery,Bus Line,Soccer Field,Bus Station,Intersection,Ice Cream Shop,Park,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,2,Indian Restaurant,Latin American Restaurant,Pet Store,Furniture / Home Store,Vietnamese Restaurant,Chinese Restaurant,Thrift / Vintage Store,Donut Shop,Dim Sum Restaurant,Diner
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,2,Auto Garage,Middle Eastern Restaurant,Breakfast Spot,Sandwich Place,Yoga Studio,Donut Shop,Discount Store,Dog Run,Doner Restaurant,Drugstore
